# 内存记忆 

## 内存记忆 

```
ConversationChatMessageHistory  会话记录
ConversationBufferMemory        内存记忆
ConversationBufferWindowMemory  可以设置记忆的条数
ConversationEntityMemory        可以实体识别
ConversationKGMemory            知识图谱
ConversationSummaryMemory       总结摘要
ConversationSummaryBufferMemory 总结摘要
ConversationVectorStoreMemory   向量存储
ConversationSQLDatabaseChainMemory
ConversationFileMemory
```

In [ ]:
from langchain_community.llms.tongyi import Tongyi
llm = Tongyi()

In [40]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("你好")
history.add_ai_message("你好?")
history.add_user_message("请问丹麦的首都是哪里?")
history.add_ai_message("哥本哈根")

In [41]:
history.messages

[HumanMessage(content='你好'),
 AIMessage(content='你好?'),
 HumanMessage(content='请问丹麦的首都是哪里?'),
 AIMessage(content='哥本哈根')]

In [42]:
llm.invoke(history.messages)

'你好! 丹麦的首都是哥本哈根。'

In [43]:
# 内存记忆
from langchain.memory import  ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")

memory.load_memory_variables({})

{'history': 'Human: 你好，我是人类！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

In [44]:
# 实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import  ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

memory.save_context({"input":"你好，我是人类!"},{"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"},{"output":"好的，我帮你找找鸡肉的做法"})
memory.save_context({"input":"我想吃鸭肉"},{"output":"好的，我帮你找找鸭肉的做法"})

memory.load_memory_variables({"input":"我想吃鸭肉"},{"output":"好的，我帮你找找鸭肉的做法"})

{'history': 'Human: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法\nHuman: 我想吃鸭肉\nAI: 好的，我帮你找找鸭肉的做法'}

## 构建记忆实体概念清单

In [45]:

from langchain.memory import ConversationEntityMemory

memory = ConversationEntityMemory(llm=llm)

_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}

memory.load_memory_variables(_input)


{'history': '', 'entities': {'胡八一': '', '王胖子': '', '雪莉杨': ''}}

In [46]:
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)

memory.load_memory_variables({"input":"铁三角是谁?"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角.\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {'胡八一': '胡八一是盗墓铁三角的一员，经常与王胖子和雪莉杨共同冒险。',
  '王胖子': '王胖子是胡八一和雪莉杨的冒险伙伴，三人并称为“盗墓铁三角”。',
  '雪莉杨': '雪莉杨是胡八一和王胖子的冒险伙伴，三人并称为“盗墓铁三角”。',
  '盗墓铁三角': ''}}

## 使用知识图谱构建记忆

In [49]:
from langchain.memory import ConversationKGMemory

memory = ConversationKGMemory(llm=llm,return_messages=True)

memory.save_context( {"input":"帮我找一下tomie"}, {"output":"对不起请问什么是tomie？"})

memory.save_context({"input":"tomie是一个培训讲师"},    {"output":"好的，我知道了。"})

In [57]:
memory.load_memory_variables({"input":"tomie是谁?"})
memory.load_memory_variables({"input":"tomie是一个培训讲师"})

{'history': []}

In [53]:
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [54]:

memory.get_knowledge_triplets("tomie最喜欢做什么事")

[]

In [55]:
memory.get_current_entities("tomie最喜欢打游戏")

['tomie']

In [59]:

memory.get_current_entities("tomie是一个培训讲师")

['tomie']

In [60]:

memory.get_knowledge_triplets("tomie是一个培训讲师")

[KnowledgeTriple(subject='Tomie', predicate='is a', object_='training instructor')]

## 总结摘要

In [61]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下张三"},
    {"output":"对不起请问谁是张三？"}
)
memory.save_context(
    {"input":"张三是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [62]:
memory.load_memory_variables({})

{'history': 'The human asked the AI to find information about Zhang San, specifying that he is a training instructor. The AI acknowledged the information and will now search accordingly.'}

In [63]:
messages = memory.chat_memory.messages
#print(messages)
memory.predict_new_summary(messages,"")

'The human requests to find information about Zhang San, specifying that he is a training instructor. The AI acknowledges the information and proceeds to search.'

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)
memory.load_memory_variables({})

# 长时记忆

通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话

In [64]:
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.dashscope import DashScopeEmbeddings

memory = ConversationBufferMemory()
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

print(memory.buffer)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    DashScopeEmbeddings()
)
FAISS.save_local(vectorstore,"test_faiss")

Human: 帮我找一下tomie
AI: 对不起请问什么是tomie？
Human: tomie是一个培训讲师
AI: 好的，我知道了。
Human: 今天他要讲一门关于RAG的课程
AI: 好的，我知道了。需要RAG的资料吗？
Human: 不需要资料了，谢谢
AI: 好的，那我就不打扰你了。


In [65]:
faiss = FAISS.load_local("test_faiss",DashScopeEmbeddings(),allow_dangerous_deserialization=True)

In [66]:
faiss.similarity_search("tomie是什么职业?")

[Document(page_content='Human: tomie是一个培训讲师'),
 Document(page_content='AI: 对不起请问什么是tomie？'),
 Document(page_content='Human: 帮我找一下tomie'),
 Document(page_content='AI: 好的，那我就不打扰你了。')]

# 链上使用记忆

## LLMChain

In [ ]:
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [35]:
template = """你是一个机器人助理。
{chat_history}
user:{human_input}
AI:"""

prompt= PromptTemplate(
    template=template,
    input_variables=["chat_history", "human_input"],
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
)


In [36]:
chain.invoke("中国的首都是哪里？")

{'human_input': '中国的首都是哪里？', 'chat_history': '', 'text': '中国的首都是北京。'}

In [37]:
chain.invoke("推荐一个旅游景点")

{'human_input': '推荐一个旅游景点',
 'chat_history': 'Human: 中国的首都是哪里？\nAI: 中国的首都是北京。',
 'text': '我推荐你去长城。长城是中国的标志性建筑之一，也是世界文化遗产，有着悠久的历史和壮丽的景色。无论是春天的花海、夏日的翠绿、秋天的金黄还是冬季的雪景，长城都有不同的魅力，会给你的旅行带来难忘的体验。同时，如果你对历史感兴趣，长城也是了解中国历史文化的重要场所。'}

In [38]:
chain.invoke("怎么去？")

{'human_input': '怎么去？',
 'chat_history': 'Human: 中国的首都是哪里？\nAI: 中国的首都是北京。\nHuman: 推荐一个旅游景点\nAI: 我推荐你去长城。长城是中国的标志性建筑之一，也是世界文化遗产，有着悠久的历史和壮丽的景色。无论是春天的花海、夏日的翠绿、秋天的金黄还是冬季的雪景，长城都有不同的魅力，会给你的旅行带来难忘的体验。同时，如果你对历史感兴趣，长城也是了解中国历史文化的重要场所。',
 'text': '去长城的具体交通方式取决于你所在的位置。如果你在北京，有几种常见的方法可以到达：\n\n1. **公交**：可以乘坐877路、919路等公交车直达八达岭长城或慕田峪长城，车程大约需要2-3小时。\n\n2. **旅游巴士**：很多旅行社提供长城的一日游服务，包括往返接送，方便快捷。\n\n3. **地铁+公交**：先乘坐地铁到德胜门或者朱辛庄站，然后转乘前往长城的公交。\n\n4. **自驾**：如果你有车或者选择租车，也可以自驾前往，但需要注意路况和停车位。\n\n5. **长城专列/索道**：对于某些长城景区如慕田峪，还有缆车或索道可以选择，可以直达长城顶部，省力且视野开阔。\n\n记得提前规划行程，查看最新的交通信息，并根据自己的时间和偏好选择最适合的方式。'}

## ConversationChain

In [67]:
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)
chain = ConversationChain(
    llm=llm,
    memory=memory
)



In [68]:
chain.invoke("给我讲一个笑话")

{'input': '给我讲一个笑话',
 'history': [HumanMessage(content='给我讲一个笑话'),
  AIMessage(content='当然，这里有一个笑话给您：为什么电脑永远不会感冒？因为它有“Windows”（窗户），但不开！笑点在于将电脑的“Windows”（微软操作系统）与建筑中的窗户进行了巧妙的双关。')],
 'response': '当然，这里有一个笑话给您：为什么电脑永远不会感冒？因为它有“Windows”（窗户），但不开！笑点在于将电脑的“Windows”（微软操作系统）与建筑中的窗户进行了巧妙的双关。'}

In [69]:
chain.invoke("这个不好笑")

{'input': '这个不好笑',
 'history': [HumanMessage(content='给我讲一个笑话'),
  AIMessage(content='当然，这里有一个笑话给您：为什么电脑永远不会感冒？因为它有“Windows”（窗户），但不开！笑点在于将电脑的“Windows”（微软操作系统）与建筑中的窗户进行了巧妙的双关。'),
  HumanMessage(content='这个不好笑'),
  AIMessage(content='哎呀，看来这个笑话没有逗笑您。那我再试试另一个：\n\n为什么数学书总是最不开心的？\n因为它总是有很多问题！\n\n这个笑话是基于一个文字游戏，将“数学书里有很多问题”与“人有很多烦恼”进行了关联。希望这个能稍微让您微笑一下。如果还是不合您的口味，我可以继续寻找其他类型的幽默。')],
 'response': '哎呀，看来这个笑话没有逗笑您。那我再试试另一个：\n\n为什么数学书总是最不开心的？\n因为它总是有很多问题！\n\n这个笑话是基于一个文字游戏，将“数学书里有很多问题”与“人有很多烦恼”进行了关联。希望这个能稍微让您微笑一下。如果还是不合您的口味，我可以继续寻找其他类型的幽默。'}

In [70]:
memory.buffer

[HumanMessage(content='给我讲一个笑话'),
 AIMessage(content='当然，这里有一个笑话给您：为什么电脑永远不会感冒？因为它有“Windows”（窗户），但不开！笑点在于将电脑的“Windows”（微软操作系统）与建筑中的窗户进行了巧妙的双关。'),
 HumanMessage(content='这个不好笑'),
 AIMessage(content='哎呀，看来这个笑话没有逗笑您。那我再试试另一个：\n\n为什么数学书总是最不开心的？\n因为它总是有很多问题！\n\n这个笑话是基于一个文字游戏，将“数学书里有很多问题”与“人有很多烦恼”进行了关联。希望这个能稍微让您微笑一下。如果还是不合您的口味，我可以继续寻找其他类型的幽默。')]

## 同一个链合并使用多个memory

In [71]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    CombinedMemory
)
from langchain.prompts import PromptTemplate


#使用CoversationSummaryMemory对对话进行总结
summay = ConversationSummaryMemory(
    llm=llm,
    input_key="input"
)
#使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(
    memory_key="history_now",
    input_key="input",
)

memory = CombinedMemory(
    memories=[summay, cov_memory],
)

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
{history}
当前对话:
{history_now}
Human:{input}
AI："""

prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["history", "history_now", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt
)




In [73]:
chain.run("2024年NBA冠军是谁")

'人类老爷，我真的不知道。'

## 多参数链增加记忆

In [74]:
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

with open("doc/NBA新闻.txt",encoding='utf-8') as f:
    text = f.read()
#切分文本
    text_splitter = CharacterTextSplitter(
        chunk_size = 200,
        chunk_overlap = 5
    )
    texts = text_splitter.split_text(text)

#使用openai的embedding
    embddings = DashScopeEmbeddings()
    #使用chroma向量存储
    docssearch = Chroma.from_texts(
        texts,
        embddings,
    )
    query = "公司有什么新策略?"
    docs = docssearch.similarity_search(query=query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [77]:
#构建问答对话链
from langchain.chains.question_answering import  load_qa_chain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
{context}
{chat_history}
Human:{human_input}
AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "chat_history", "human_input"],
)

#使用ConversationBufferMemory对对话进行缓存 
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    return_messages=True,
)

#加载对话链
chain = load_qa_chain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    chain_type="stuff"
)

# chain.run("2024年NBA冠军是谁")
# chain({"input_documents":docs,"human_input":"公司的营销策略是什么？"})
chain({"input_documents":docs,"human_input":"2024年NBA冠军是谁？"})


{'input_documents': [Document(page_content='在2024年NBA总决赛的最后一场比赛中，丹佛掘金在主场以94-89击败热火，以总比分4-1成功夺得NBA总冠军。这是球队历史上第一次获得总冠军，也是球队自1976年进入NBA以来的最佳成绩。丹佛掘金队的成功主要归功于他们的领袖球员约基奇的出色表现。\n在总决赛的五场比赛中，约基奇展现出了惊人的统治力。他场均贡献30.2分、14个篮板和7.2次助攻，成为球队得分、篮板和助攻的核心。约基奇在进攻端展现出了全面的技术，他的得分能力和篮板能力让热火队无可奈何。同时，他还展现出了出色的组织能力，为球队创造了很多得分机会。\n在总决赛的最后一场比赛中，约基奇更是发挥出色。他在关键时刻承担责任，不仅在进攻端贡献了关键得分，还在防守端起到了重要作用。他的领导能力和稳定性为球队赢得了决胜的胜利。\n约基奇荣获总决赛最有价值球员（MVP）毫无悬念。他在总决赛中的出色表现让他成为了不可或缺的球队核心，也让他获得了职业生涯中的首个总冠军。这一荣誉不仅是对他个人努力的认可，也是对他带领球队取得成功的肯定。\n随着约基奇的崛起，丹佛掘金队在过去几个赛季中逐渐崭露头角。他的全面发展和领导能力使他成为了球队的核心和灵魂人物。通过这次总决赛的胜利，约基奇不仅实现了自己的篮球梦想，也为球队带来了无比的荣耀。\n约基奇带领丹佛掘金赢得2024年NBA总冠军，并凭借出色的表现获得总决赛最有价值球员（MVP）的荣誉。他在总决赛期间的统治力和全面能力使他成为球队的核心，同时也展现了他的领导才能。这次胜利不仅是约基奇个人职业生涯的里程碑，也是丹佛掘金队迈向更高荣耀的关键一步。随着约基奇的领导，丹佛掘金队有望在未来继续取得更多的成功。'),
  Document(page_content='在2024年NBA总决赛的最后一场比赛中，丹佛掘金在主场以94-89击败热火，以总比分4-1成功夺得NBA总冠军。这是球队历史上第一次获得总冠军，也是球队自1976年进入NBA以来的最佳成绩。丹佛掘金队的成功主要归功于他们的领袖球员约基奇的出色表现。\n在总决赛的五场比赛中，约基奇展现出了惊人的统治力。他场均贡献30.2分、14个篮板和7.2次助攻，成为球队得分、篮板和助攻的核心。约基奇在进攻端展现出了全面的技术，他的得分能力和篮板能力让